# Deploying the Model of the Best Tuning Job of an Autopilot Experiment using the SageMaker Python SDK

<img align="left" width="130" src="https://raw.githubusercontent.com/PacktPublishing/Amazon-SageMaker-Cookbook/master/Extra/cover-small-padded.png"/>

This notebook contains the code to help readers work through one of the recipes of the book [Machine Learning with Amazon SageMaker Cookbook: 80 proven recipes for data scientists and developers to perform ML experiments and deployments](https://www.amazon.com/Machine-Learning-Amazon-SageMaker-Cookbook/dp/1800567030)

### How to do it...

In [ ]:
%store -r autopilot_job_name

In [ ]:
%store -r s3_bucket_name
s3_bucket_name

In [ ]:
%store -r prefix
prefix

In [ ]:
import sagemaker
session = sagemaker.Session()

In [ ]:
autopilot_job = sagemaker.AutoML.attach(autopilot_job_name, 
                                        sagemaker_session=session)
endpoint_name = 'endpoint-' + autopilot_job_name

In [ ]:
%%time

autopilot_job.deploy(
    initial_instance_count=1, 
    instance_type='ml.m5.large', 
    endpoint_name=endpoint_name)

In [ ]:
from sagemaker.predictor import Predictor
predictor = Predictor(endpoint_name, 
                      sagemaker_session=session)

In [ ]:
!mkdir -p tmp

In [ ]:
!aws s3 cp s3://{s3_bucket_name}/{prefix}/test_data.csv tmp/test_data.csv 

In [ ]:
import pandas as pd

test_data = pd.read_csv("tmp/test_data.csv")

In [ ]:
test_data

In [ ]:
test_data_without_label = test_data.drop(columns=['label'])
test_data_without_label

In [ ]:
test_rows = test_data_without_label[0:5].to_csv(index=False, header=False)
test_rows

In [ ]:
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer

predictor.serializer = CSVSerializer()
predictor.deserializer = JSONDeserializer()

predictor.predict(test_rows)

In [ ]:
predictor.delete_endpoint()